# Notebook 18: Safety Validation and Testing for AV Perception

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_4_Uncertainty_Estimation_and_Validation/notebooks/18_Safety_Validation_and_Testing.ipynb)

**Session 4: Uncertainty Estimation and Validation**  
**Duration:** 25 minutes

## Learning Objectives
- Understand validation challenges for ML-based AV perception
- Learn scenario-based testing (Pegasus 6-layer model)
- Explore simulation-based validation approaches
- Understand X-in-the-Loop testing (SIL, HIL, VIL)
- Learn statistical validation requirements
- Design a complete validation strategy

---

## Introduction

**The Validation Challenge:**

Traditional software: Verify against requirements, test all code paths.

**ML-based perception:**
- Infinite possible inputs (weather, lighting, objects, ...)
- No explicit rules to verify
- Probabilistic outputs
- Corner cases are safety-critical

**Kalra & Paddock (2016): "Driving to Safety"**
- To prove 20% better than human drivers (95% confidence)
- Need to drive **275 million miles** without failure
- At 25 mph, 24/7: Would take **500 years** for one vehicle!

**Solution:** Combination of approaches
1. Scenario-based testing
2. Simulation
3. Accelerated testing
4. Statistical methods
5. Field operational tests

In [ ]:
# Setup
!pip install -q matplotlib seaborn numpy scipy pandas plotly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
from dataclasses import dataclass
from typing import List, Dict, Tuple
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

np.random.seed(42)

## 1. Scenario-Based Testing

### 1.1 The Pegasus 6-Layer Model

**Pegasus Project:** German research project for safety validation of AVs

**6 Layers of Scenario Abstraction:**

1. **Layer 1 - Concrete Scenario:** Fully specified, single instance
   - "On Street X, at 3pm on June 1st, sunny weather, pedestrian crosses from left"
   - Can be replayed exactly
   - Used for debugging specific failures

2. **Layer 2 - Concrete Scenario with Ranges:** Parameters with ranges
   - "Pedestrian crosses from left, speed 1.0-1.5 m/s, distance 10-15m"
   - Enables parameter sweeps

3. **Layer 3 - Functional Scenario:** Described in functional terms
   - "Pedestrian crossing scenario"
   - Road type: 2-lane urban
   - Actor: pedestrian
   - Maneuver: crossing perpendicular to road

4. **Layer 4 - Logical Scenario:** All possible parameter combinations
   - All variations of pedestrian crossing
   - Different speeds, angles, occlusions, lighting, weather, ...
   - Defines the **search space** for testing

5. **Layer 5 - Scenario Category:** General categories
   - "Vulnerable road user interaction"
   - Groups related scenarios

6. **Layer 6 - Operational Design Domain (ODD):** Full operating envelope
   - "Urban environment, <50 km/h, daylight, dry roads"
   - Defines where system is designed to operate

In [ ]:
@dataclass
class ConcreteScenario:
    """Layer 1: Concrete scenario with specific parameters."""
    name: str
    road_type: str
    weather: str
    lighting: str
    pedestrian_speed: float  # m/s
    pedestrian_distance: float  # m
    ego_speed: float  # km/h
    occlusion: bool
    
    def describe(self):
        return (f"Scenario: {self.name}\n"
               f"  Road: {self.road_type}\n"
               f"  Weather: {self.weather}, Lighting: {self.lighting}\n"
               f"  Pedestrian: {self.pedestrian_speed} m/s at {self.pedestrian_distance}m\n"
               f"  Ego speed: {self.ego_speed} km/h\n"
               f"  Occlusion: {'Yes' if self.occlusion else 'No'}")

@dataclass
class FunctionalScenario:
    """Layer 3: Functional scenario description."""
    name: str
    scenario_type: str
    road_type: List[str]
    weather_conditions: List[str]
    lighting_conditions: List[str]
    pedestrian_speed_range: Tuple[float, float]
    pedestrian_distance_range: Tuple[float, float]
    ego_speed_range: Tuple[float, float]
    
    def generate_concrete_scenarios(self, n_samples=10):
        """Generate concrete scenarios from functional description."""
        scenarios = []
        for i in range(n_samples):
            scenario = ConcreteScenario(
                name=f"{self.name}_{i+1}",
                road_type=np.random.choice(self.road_type),
                weather=np.random.choice(self.weather_conditions),
                lighting=np.random.choice(self.lighting_conditions),
                pedestrian_speed=np.random.uniform(*self.pedestrian_speed_range),
                pedestrian_distance=np.random.uniform(*self.pedestrian_distance_range),
                ego_speed=np.random.uniform(*self.ego_speed_range),
                occlusion=np.random.choice([True, False])
            )
            scenarios.append(scenario)
        return scenarios

# Example functional scenario
pedestrian_crossing = FunctionalScenario(
    name="Pedestrian_Crossing",
    scenario_type="VRU_Interaction",
    road_type=["urban_2lane", "residential"],
    weather_conditions=["clear", "rain", "fog"],
    lighting_conditions=["day", "night", "dusk"],
    pedestrian_speed_range=(0.5, 2.0),  # m/s
    pedestrian_distance_range=(5.0, 30.0),  # m
    ego_speed_range=(20.0, 50.0)  # km/h
)

# Generate concrete scenarios
concrete_scenarios = pedestrian_crossing.generate_concrete_scenarios(n_samples=5)

print("Generated Concrete Scenarios:\n" + "="*60)
for scenario in concrete_scenarios:
    print(scenario.describe())
    print("-"*60)

In [ ]:
# Visualize scenario space
def visualize_scenario_space():
    """Visualize the scenario parameter space."""
    
    # Generate larger sample
    scenarios = pedestrian_crossing.generate_concrete_scenarios(n_samples=200)
    
    # Extract parameters
    ped_speeds = [s.pedestrian_speed for s in scenarios]
    ped_distances = [s.pedestrian_distance for s in scenarios]
    ego_speeds = [s.ego_speed for s in scenarios]
    
    # Create difficulty score (higher = more challenging)
    difficulty = []
    for s in scenarios:
        # Difficulty increases with:
        # - Higher ego speed
        # - Shorter pedestrian distance
        # - Higher pedestrian speed
        # - Occlusion, bad weather/lighting
        score = 0
        score += (s.ego_speed / 50) * 30  # 0-30 points
        score += (1 - s.pedestrian_distance / 30) * 25  # 0-25 points
        score += (s.pedestrian_speed / 2) * 15  # 0-15 points
        if s.occlusion:
            score += 15
        if s.weather != 'clear':
            score += 10
        if s.lighting != 'day':
            score += 5
        difficulty.append(score)
    
    # Plot
    fig = plt.figure(figsize=(16, 10))
    
    # 3D scatter plot
    ax1 = fig.add_subplot(221, projection='3d')
    scatter = ax1.scatter(ped_speeds, ped_distances, ego_speeds, 
                         c=difficulty, cmap='RdYlGn_r', s=50, alpha=0.6)
    ax1.set_xlabel('Pedestrian Speed (m/s)', fontsize=10)
    ax1.set_ylabel('Pedestrian Distance (m)', fontsize=10)
    ax1.set_zlabel('Ego Speed (km/h)', fontsize=10)
    ax1.set_title('Scenario Parameter Space\n(Color = Difficulty)', fontsize=12, fontweight='bold')
    plt.colorbar(scatter, ax=ax1, label='Difficulty Score')
    
    # 2D heatmap: ego speed vs pedestrian distance
    ax2 = fig.add_subplot(222)
    H, xedges, yedges = np.histogram2d(ego_speeds, ped_distances, bins=20)
    im = ax2.imshow(H.T, origin='lower', cmap='Blues', aspect='auto',
                   extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    ax2.set_xlabel('Ego Speed (km/h)', fontsize=11)
    ax2.set_ylabel('Pedestrian Distance (m)', fontsize=11)
    ax2.set_title('Coverage: Ego Speed vs Distance', fontsize=12, fontweight='bold')
    plt.colorbar(im, ax=ax2, label='Number of Scenarios')
    
    # Difficulty distribution
    ax3 = fig.add_subplot(223)
    ax3.hist(difficulty, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    ax3.axvline(np.percentile(difficulty, 90), color='red', linestyle='--', 
               linewidth=2, label='90th percentile (critical scenarios)')
    ax3.set_xlabel('Difficulty Score', fontsize=11)
    ax3.set_ylabel('Number of Scenarios', fontsize=11)
    ax3.set_title('Scenario Difficulty Distribution', fontsize=12, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Category breakdown
    ax4 = fig.add_subplot(224)
    weather_counts = pd.Series([s.weather for s in scenarios]).value_counts()
    lighting_counts = pd.Series([s.lighting for s in scenarios]).value_counts()
    
    x = np.arange(len(weather_counts))
    width = 0.35
    
    # Only plot if we have lighting data to match
    ax4_2 = ax4.twinx()
    
    bars1 = ax4.bar(x - width/2, weather_counts.values, width, 
                   label='Weather', alpha=0.7, color='skyblue', edgecolor='black')
    ax4.set_ylabel('Weather Conditions Count', fontsize=11)
    ax4.set_xticks(x)
    ax4.set_xticklabels(weather_counts.index, rotation=45)
    ax4.set_title('Scenario Conditions Coverage', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Lighting on secondary axis
    x2 = np.arange(len(lighting_counts))
    bars2 = ax4_2.bar(x2 + width/2, lighting_counts.values, width,
                     label='Lighting', alpha=0.7, color='orange', edgecolor='black')
    ax4_2.set_ylabel('Lighting Conditions Count', fontsize=11)
    
    # Combined legend
    lines1, labels1 = ax4.get_legend_handles_labels()
    lines2, labels2 = ax4_2.get_legend_handles_labels()
    ax4.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("\n" + "="*70)
    print("SCENARIO SPACE ANALYSIS")
    print("="*70)
    print(f"\nTotal scenarios generated: {len(scenarios)}")
    print(f"\nDifficulty Score Statistics:")
    print(f"  Mean: {np.mean(difficulty):.2f}")
    print(f"  Std:  {np.std(difficulty):.2f}")
    print(f"  Min:  {np.min(difficulty):.2f}")
    print(f"  Max:  {np.max(difficulty):.2f}")
    print(f"  90th percentile: {np.percentile(difficulty, 90):.2f}")
    print(f"\nCritical scenarios (>90th percentile): {sum(d > np.percentile(difficulty, 90) for d in difficulty)}")
    
    print("\n🎯 Validation Strategy:")
    print("  1. Test ALL critical scenarios (top 10%)")
    print("  2. Sample moderate scenarios (middle 80%)")
    print("  3. Include some easy scenarios (bottom 10%) for baseline")
    print("  4. Ensure coverage across all weather/lighting conditions")

visualize_scenario_space()

## 2. Simulation-Based Validation

### 2.1 Why Simulation?

**Advantages:**
- ✅ Safe: No risk to people or vehicles
- ✅ Repeatable: Exact scenario replay
- ✅ Scalable: Run 24/7, parallel instances
- ✅ Controllable: Precisely set parameters
- ✅ Accelerated: Test rare scenarios
- ✅ Measurable: Full ground truth

**Challenges:**
- ❌ Sim-to-real gap: Sensors, physics, actor behavior
- ❌ Validation of simulator: How do we know sim is correct?
- ❌ Corner case modeling: Hard to simulate unknown scenarios

### 2.2 Popular AV Simulators

**CARLA (Open-source):**
- Based on Unreal Engine
- Physics simulation
- Sensor suite (cameras, LiDAR, radar)
- Python API
- Used for: Perception, planning, control testing

**LGSVL (Open-source, now Apollo):**
- Unity-based
- High-fidelity sensor simulation
- ROS integration
- Cloud deployment

**MetaDrive (Research):**
- Lightweight, fast
- Procedural generation
- RL-focused

**Commercial:** IPG CarMaker, dSPACE, ANSYS, rFpro, etc.

### 2.3 Coverage Metrics

In [ ]:
class CoverageAnalyzer:
    """
    Analyze test coverage for scenario-based testing.
    """
    
    def __init__(self, name: str):
        self.name = name
        self.coverage_bins = {}
        self.total_bins = 0
        
    def define_coverage_space(self, parameter_bins: Dict[str, int]):
        """
        Define the parameter space for coverage analysis.
        
        Args:
            parameter_bins: Dict of parameter name -> number of bins
        """
        self.parameter_bins = parameter_bins
        
        # Calculate total number of bins (combinations)
        self.total_bins = 1
        for n_bins in parameter_bins.values():
            self.total_bins *= n_bins
        
        print(f"Coverage space defined: {self.total_bins:,} total bins")
        print(f"Parameters: {list(parameter_bins.keys())}")
        
    def compute_coverage(self, tested_scenarios: List[ConcreteScenario]):
        """
        Compute coverage percentage from tested scenarios.
        """
        # Simplified: count unique combinations of discretized parameters
        covered_bins = set()
        
        for scenario in tested_scenarios:
            # Create bin identifier (simplified)
            bin_id = (
                scenario.weather,
                scenario.lighting,
                int(scenario.pedestrian_speed * 2),  # Bin by 0.5 m/s
                int(scenario.pedestrian_distance / 5),  # Bin by 5m
                int(scenario.ego_speed / 10),  # Bin by 10 km/h
                scenario.occlusion
            )
            covered_bins.add(bin_id)
        
        coverage_pct = len(covered_bins) / self.total_bins * 100
        
        return coverage_pct, len(covered_bins)
    
    def visualize_coverage_growth(self, max_scenarios=1000, step=50):
        """
        Visualize how coverage grows with number of test scenarios.
        """
        n_scenarios_list = list(range(step, max_scenarios + 1, step))
        coverage_list = []
        
        for n in n_scenarios_list:
            scenarios = pedestrian_crossing.generate_concrete_scenarios(n)
            coverage, _ = self.compute_coverage(scenarios)
            coverage_list.append(coverage)
        
        # Plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Coverage growth
        ax1.plot(n_scenarios_list, coverage_list, 'b-', linewidth=2, marker='o')
        ax1.axhline(y=80, color='green', linestyle='--', linewidth=2, label='Target: 80%')
        ax1.axhline(y=95, color='orange', linestyle='--', linewidth=2, label='Goal: 95%')
        ax1.set_xlabel('Number of Test Scenarios', fontsize=12)
        ax1.set_ylabel('Coverage (%)', fontsize=12)
        ax1.set_title('Test Coverage Growth', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Diminishing returns
        marginal_coverage = [0] + [coverage_list[i] - coverage_list[i-1] 
                                   for i in range(1, len(coverage_list))]
        ax2.bar(n_scenarios_list, marginal_coverage, width=step*0.8, 
               color='steelblue', alpha=0.7, edgecolor='black')
        ax2.set_xlabel('Number of Test Scenarios', fontsize=12)
        ax2.set_ylabel('Marginal Coverage Gain (%)', fontsize=12)
        ax2.set_title('Diminishing Returns in Coverage', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        # Find scenarios needed for thresholds
        for threshold in [80, 90, 95]:
            for i, cov in enumerate(coverage_list):
                if cov >= threshold:
                    print(f"To reach {threshold}% coverage: ~{n_scenarios_list[i]} scenarios needed")
                    break

# Define coverage space
analyzer = CoverageAnalyzer("Pedestrian Crossing")
analyzer.define_coverage_space({
    'weather': 3,  # clear, rain, fog
    'lighting': 3,  # day, night, dusk
    'pedestrian_speed': 4,  # binned
    'pedestrian_distance': 6,  # binned
    'ego_speed': 4,  # binned
    'occlusion': 2  # yes/no
})

print("\n")
analyzer.visualize_coverage_growth(max_scenarios=1000, step=50)

## 3. X-in-the-Loop Testing

**Progressive validation strategy from simulation to real world:**

### 3.1 SIL (Software-in-the-Loop)

**What:** Software runs entirely in simulation

**Setup:**
- Simulated sensors → Perception → Planning → Control → Simulated vehicle
- All components are software

**Advantages:**
- ✅ Fastest iteration
- ✅ Cheapest
- ✅ Unlimited scenarios

**Use for:**
- Algorithm development
- Functional testing
- Coverage analysis

### 3.2 HIL (Hardware-in-the-Loop)

**What:** Real hardware (ECUs, sensors) connected to simulation

**Setup:**
- Real sensors (camera, LiDAR) → Real compute → Simulated vehicle dynamics
- Or: Sensor simulation → Real compute hardware → Simulated actuators

**Advantages:**
- ✅ Tests real hardware interfaces
- ✅ Tests real-time performance
- ✅ Detects hardware-specific issues

**Use for:**
- Integration testing
- Timing validation
- ECU testing

### 3.3 VIL (Vehicle-in-the-Loop)

**What:** Real vehicle in controlled environment (proving ground)

**Setup:**
- Real vehicle, real sensors, real actuators
- Controlled test track
- Soft targets (inflatable pedestrians, etc.)

**Advantages:**
- ✅ Real vehicle dynamics
- ✅ Real sensor data
- ✅ Safety-critical scenarios possible

**Use for:**
- Final validation before public roads
- Rare/dangerous scenarios
- Regulatory testing

### 3.4 FOT (Field Operational Tests)

**What:** Real-world driving on public roads

**Requirements:**
- Safety driver
- Extensive monitoring
- Regulatory approval

**Use for:**
- Final validation
- Long-tail scenario discovery
- Performance in real conditions

In [ ]:
def visualize_testing_pyramid():
    """
    Visualize the testing strategy pyramid.
    """
    
    # Define pyramid levels
    levels = [
        {'name': 'FOT\n(Field Tests)', 'scenarios': 100, 'cost': 10000, 'fidelity': 100},
        {'name': 'VIL\n(Proving Ground)', 'scenarios': 1000, 'cost': 1000, 'fidelity': 90},
        {'name': 'HIL\n(Hardware-in-Loop)', 'scenarios': 10000, 'cost': 100, 'fidelity': 70},
        {'name': 'SIL\n(Simulation)', 'scenarios': 1000000, 'cost': 1, 'fidelity': 50},
    ]
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 6))
    
    # Pyramid 1: Number of scenarios
    ax = axes[0]
    y_pos = np.arange(len(levels))
    widths = [l['scenarios'] for l in levels]
    colors = ['red', 'orange', 'yellow', 'lightgreen']
    
    for i, (level, width, color) in enumerate(zip(levels, widths, colors)):
        ax.barh(i, width, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        # Add text
        ax.text(width/2, i, f"{level['name']}\n{width:,} scenarios", 
               ha='center', va='center', fontsize=10, fontweight='bold')
    
    ax.set_yticks([])
    ax.set_xlabel('Number of Test Scenarios (log scale)', fontsize=11)
    ax.set_xscale('log')
    ax.set_title('Testing Pyramid: Volume', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Pyramid 2: Cost per scenario
    ax = axes[1]
    costs = [l['cost'] for l in levels]
    
    for i, (level, cost, color) in enumerate(zip(levels, costs, colors)):
        ax.barh(i, cost, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        ax.text(cost/2, i, f"${cost:,}/scenario",
               ha='center', va='center', fontsize=10, fontweight='bold')
    
    ax.set_yticks([])
    ax.set_xlabel('Relative Cost per Scenario (log scale)', fontsize=11)
    ax.set_xscale('log')
    ax.set_title('Testing Pyramid: Cost', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Pyramid 3: Fidelity
    ax = axes[2]
    fidelities = [l['fidelity'] for l in levels]
    
    for i, (level, fid, color) in enumerate(zip(levels, fidelities, colors)):
        ax.barh(i, fid, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        ax.text(fid/2, i, f"{fid}% fidelity",
               ha='center', va='center', fontsize=10, fontweight='bold')
    
    ax.set_yticks([])
    ax.set_xlabel('Fidelity to Real World (%)', fontsize=11)
    ax.set_title('Testing Pyramid: Fidelity', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_xlim([0, 100])
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*70)
    print("TESTING STRATEGY PYRAMID")
    print("="*70)
    print(f"\n{'Level':<25} {'Scenarios':<15} {'Cost/Scenario':<15} {'Fidelity'}")
    print("-"*70)
    for level in levels:
        print(f"{level['name'].replace(chr(10), ' '):<25} {level['scenarios']:<15,} "
              f"${level['cost']:<14,} {level['fidelity']}%")
    
    print("\n" + "="*70)
    print("\n🎯 Validation Strategy:")
    print("\n1. SIL (Simulation):")
    print("   • Run MILLIONS of scenarios")
    print("   • Cover entire parameter space")
    print("   • Find algorithmic issues")
    print("\n2. HIL (Hardware-in-Loop):")
    print("   • Run THOUSANDS of scenarios")
    print("   • Focus on critical/corner cases")
    print("   • Validate real-time performance")
    print("\n3. VIL (Vehicle-in-Loop):")
    print("   • Run HUNDREDS of scenarios")
    print("   • Test dangerous scenarios safely")
    print("   • Validate complete system")
    print("\n4. FOT (Field Tests):")
    print("   • Run ~100 critical scenarios")
    print("   • Plus general driving for long-tail discovery")
    print("   • Final confidence before deployment")
    print("\n💡 Total testing: 1M+ SIL + 10K HIL + 1K VIL + 100 FOT scenarios")

visualize_testing_pyramid()

## 4. Statistical Validation Requirements

### 4.1 The Kalra & Paddock Analysis

**Question:** How many miles to prove AV is safer than humans?

**Assumptions:**
- Human fatality rate: ~1 per 100 million miles
- Want to prove AV is 20% better with 95% confidence

**Answer:** **275 million failure-free miles**

**At 25 mph, 24/7:** Would take ~500 years for one vehicle!

### 4.2 Confidence Intervals for Rare Events

In [ ]:
def compute_statistical_evidence(target_failure_rate, confidence_level=0.95, 
                                 improvement_factor=1.2):
    """
    Compute miles needed to demonstrate safety with statistical confidence.
    
    Uses Poisson distribution for rare events.
    
    Args:
        target_failure_rate: Failures per mile (e.g., 1/100M for fatalities)
        confidence_level: Statistical confidence (e.g., 0.95)
        improvement_factor: How much better than target (e.g., 1.2 = 20% better)
    """
    # Target rate for AV (better than baseline)
    av_target_rate = target_failure_rate / improvement_factor
    
    # For zero failures observed, upper confidence bound:
    # λ_upper = -ln(1 - confidence) / n_miles
    # We want: λ_upper <= av_target_rate
    # Therefore: n_miles >= -ln(1 - confidence) / av_target_rate
    
    miles_needed = -np.log(1 - confidence_level) / av_target_rate
    
    return miles_needed

def visualize_statistical_requirements():
    """
    Visualize miles needed for different safety metrics.
    """
    
    # Different safety metrics
    metrics = [
        {'name': 'Fatality', 'baseline_rate': 1/100e6, 'critical': True},
        {'name': 'Injury accident', 'baseline_rate': 1/1e6, 'critical': True},
        {'name': 'Any accident', 'baseline_rate': 1/100e3, 'critical': False},
        {'name': 'Critical intervention', 'baseline_rate': 1/10e3, 'critical': False},
    ]
    
    confidence_levels = [0.90, 0.95, 0.99]
    improvement_factors = [1.2, 1.5, 2.0]  # 20%, 50%, 100% better
    
    # Compute for base case: 95% confidence, 20% improvement
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    # Plot 1: Miles needed for different metrics
    ax = axes[0]
    miles_needed = []
    colors_list = ['darkred', 'red', 'orange', 'yellow']
    
    for metric in metrics:
        miles = compute_statistical_evidence(metric['baseline_rate'], 
                                            confidence_level=0.95,
                                            improvement_factor=1.2)
        miles_needed.append(miles / 1e6)  # Convert to millions
    
    bars = ax.barh(range(len(metrics)), miles_needed, 
                   color=colors_list, alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_yticks(range(len(metrics)))
    ax.set_yticklabels([m['name'] for m in metrics])
    ax.set_xlabel('Miles Needed (Millions)', fontsize=11)
    ax.set_title('Statistical Evidence Required\n(95% confidence, 20% improvement)', 
                fontsize=12, fontweight='bold')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add values
    for i, (bar, miles) in enumerate(zip(bars, miles_needed)):
        ax.text(miles, i, f'  {miles:.0f}M miles', 
               va='center', fontsize=9, fontweight='bold')
    
    # Plot 2: Effect of confidence level
    ax = axes[1]
    metric = metrics[0]  # Use fatality as example
    
    miles_by_conf = []
    for conf in confidence_levels:
        miles = compute_statistical_evidence(metric['baseline_rate'],
                                            confidence_level=conf,
                                            improvement_factor=1.2)
        miles_by_conf.append(miles / 1e6)
    
    bars = ax.bar([f"{int(c*100)}%" for c in confidence_levels], miles_by_conf,
                  color='steelblue', alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Miles Needed (Millions)', fontsize=11)
    ax.set_xlabel('Confidence Level', fontsize=11)
    ax.set_title('Effect of Confidence Level\n(Fatality metric, 20% improvement)',
                fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, miles in zip(bars, miles_by_conf):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{miles:.0f}M',
               ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Plot 3: Effect of improvement factor
    ax = axes[2]
    miles_by_improv = []
    for improv in improvement_factors:
        miles = compute_statistical_evidence(metric['baseline_rate'],
                                            confidence_level=0.95,
                                            improvement_factor=improv)
        miles_by_improv.append(miles / 1e6)
    
    bars = ax.bar([f"{int((f-1)*100)}%" for f in improvement_factors], 
                  miles_by_improv,
                  color='green', alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Miles Needed (Millions)', fontsize=11)
    ax.set_xlabel('Improvement over Baseline', fontsize=11)
    ax.set_title('Effect of Target Improvement\n(Fatality metric, 95% confidence)',
                fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, miles in zip(bars, miles_by_improv):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{miles:.0f}M',
               ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Plot 4: Time required
    ax = axes[3]
    
    # Compute time for different fleet sizes
    target_miles = miles_needed[0] * 1e6  # Fatality metric
    fleet_sizes = [1, 10, 100, 1000]
    avg_speed_mph = 25
    hours_per_day = 24
    
    years_needed = []
    for fleet_size in fleet_sizes:
        miles_per_day = fleet_size * avg_speed_mph * hours_per_day
        days_needed = target_miles / miles_per_day
        years = days_needed / 365
        years_needed.append(years)
    
    bars = ax.bar([f"{fs} vehicle{'s' if fs > 1 else ''}" for fs in fleet_sizes],
                  years_needed,
                  color='purple', alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Years Required', fontsize=11)
    ax.set_xlabel('Fleet Size', fontsize=11)
    ax.set_title(f'Time to Collect {miles_needed[0]:.0f}M Miles\n(25 mph average, 24/7 operation)',
                fontsize=12, fontweight='bold')
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, years in zip(bars, years_needed):
        height = bar.get_height()
        if years >= 1:
            label = f'{years:.0f} years'
        else:
            label = f'{years*12:.0f} months'
        ax.text(bar.get_x() + bar.get_width()/2., height,
               label,
               ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n" + "="*80)
    print("STATISTICAL VALIDATION REQUIREMENTS (Kalra & Paddock Analysis)")
    print("="*80)
    print(f"\n{'Metric':<25} {'Baseline Rate':<20} {'Miles Needed (95% conf, 20% better)'}")
    print("-"*80)
    
    for metric, miles in zip(metrics, miles_needed):
        rate_str = f"1 per {int(1/metric['baseline_rate']):,} miles"
        print(f"{metric['name']:<25} {rate_str:<20} {miles:.1f} million miles")
    
    print("\n" + "="*80)
    print("\n🎯 Key Insights:")
    print("\n1. Pure statistical proof for fatalities is IMPRACTICAL")
    print(f"   • Need {miles_needed[0]:.0f}M miles = {years_needed[0]:.0f} years for 1 vehicle")
    print(f"   • Even with 100 vehicles: {years_needed[2]:.1f} years")
    print("\n2. Must use SURROGATE metrics:")
    print("   • Critical interventions (easier to measure)")
    print("   • Near-miss events")
    print("   • Disengagements")
    print("\n3. Complementary evidence needed:")
    print("   • Simulation validation (millions of scenarios)")
    print("   • Proving ground tests")
    print("   • Safety argument based on design")
    print("   • Continuous monitoring in operation")

visualize_statistical_requirements()

## 5. Validation Metrics for AV Perception

### 5.1 Standard ML Metrics

**For object detection:**
- Precision, Recall, F1-score
- mAP (mean Average Precision)
- IoU (Intersection over Union)

**For classification:**
- Accuracy
- Confusion matrix
- ROC curve, AUC

### 5.2 Safety-Specific Metrics

**For AV perception, we prioritize:**

1. **Recall (Sensitivity):** Don't miss critical objects!
   - False negatives = potentially fatal
   - Especially for pedestrians, cyclists

2. **False Negative Rate at critical distances:**
   - Must detect pedestrians at >30m in urban scenarios
   - Must detect vehicles at >100m on highway

3. **Worst-case performance:**
   - 99th percentile latency
   - Performance in adverse conditions

4. **Uncertainty quality:**
   - Calibration (ECE)
   - Uncertainty on OOD inputs

5. **Temporal consistency:**
   - Track stability
   - No flickering detections

In [ ]:
def safety_metric_comparison():
    """
    Compare standard ML metrics vs safety-oriented metrics.
    """
    
    # Simulated detector performance
    scenarios = [
        {
            'name': 'Model A: High Accuracy',
            'TP': 950, 'FP': 50, 'FN': 50, 'TN': 950,
            'description': 'Balanced'
        },
        {
            'name': 'Model B: High Precision',
            'TP': 900, 'FP': 10, 'FN': 100, 'TN': 990,
            'description': 'Conservative (many false negatives!)'
        },
        {
            'name': 'Model C: High Recall',
            'TP': 980, 'FP': 200, 'FN': 20, 'TN': 800,
            'description': 'Aggressive (few false negatives)'
        },
    ]
    
    # Compute metrics
    for scenario in scenarios:
        TP, FP, FN, TN = scenario['TP'], scenario['FP'], scenario['FN'], scenario['TN']
        
        scenario['accuracy'] = (TP + TN) / (TP + FP + FN + TN)
        scenario['precision'] = TP / (TP + FP) if (TP + FP) > 0 else 0
        scenario['recall'] = TP / (TP + FN) if (TP + FN) > 0 else 0
        scenario['f1'] = 2 * (scenario['precision'] * scenario['recall']) / \
                        (scenario['precision'] + scenario['recall']) if \
                        (scenario['precision'] + scenario['recall']) > 0 else 0
        scenario['fnr'] = FN / (TP + FN) if (TP + FN) > 0 else 0
        scenario['fpr'] = FP / (FP + TN) if (FP + TN) > 0 else 0
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Standard metrics
    ax = axes[0, 0]
    models = [s['name'].split(':')[0] for s in scenarios]
    x = np.arange(len(models))
    width = 0.2
    
    ax.bar(x - width, [s['accuracy'] for s in scenarios], width, 
          label='Accuracy', alpha=0.8, edgecolor='black')
    ax.bar(x, [s['precision'] for s in scenarios], width,
          label='Precision', alpha=0.8, edgecolor='black')
    ax.bar(x + width, [s['recall'] for s in scenarios], width,
          label='Recall', alpha=0.8, edgecolor='black')
    
    ax.set_ylabel('Score', fontsize=11)
    ax.set_title('Standard ML Metrics', fontsize=12, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(models)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1])
    
    # Plot 2: Safety-critical metrics
    ax = axes[0, 1]
    
    fnrs = [s['fnr'] for s in scenarios]
    fprs = [s['fpr'] for s in scenarios]
    
    bars1 = ax.bar(x - width/2, fnrs, width, label='FNR (⚠️ Safety Critical!)',
                  color='red', alpha=0.7, edgecolor='black', linewidth=2)
    bars2 = ax.bar(x + width/2, fprs, width, label='FPR (Comfort)',
                  color='orange', alpha=0.7, edgecolor='black', linewidth=2)
    
    ax.set_ylabel('Error Rate', fontsize=11)
    ax.set_title('Safety-Critical Metrics (Lower is Better)', fontsize=12, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(models)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add target line
    ax.axhline(y=0.05, color='green', linestyle='--', linewidth=2, label='Target: <5%')
    
    # Plot 3: Confusion matrices
    for i, scenario in enumerate(scenarios[:2]):
        ax = axes[1, i]
        conf_matrix = np.array([[scenario['TN'], scenario['FP']],
                               [scenario['FN'], scenario['TP']]])
        
        im = ax.imshow(conf_matrix, cmap='RdYlGn', alpha=0.7, vmin=0, vmax=1000)
        
        for r in range(2):
            for c in range(2):
                text = ax.text(c, r, f'{conf_matrix[r, c]}',
                             ha="center", va="center", color="black",
                             fontsize=16, fontweight='bold')
        
        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        ax.set_xticklabels(['Pred Neg', 'Pred Pos'])
        ax.set_yticklabels(['True Neg', 'True Pos'])
        ax.set_title(f"{scenario['name']}\nFNR={scenario['fnr']:.1%}",
                    fontsize=11, fontweight='bold')
        
        # Highlight FN in red
        if scenario['FN'] > 50:
            for spine in ax.spines.values():
                spine.set_edgecolor('red')
                spine.set_linewidth(4)
    
    plt.tight_layout()
    plt.show()
    
    # Print analysis
    print("\n" + "="*80)
    print("SAFETY METRIC ANALYSIS")
    print("="*80)
    print(f"\n{'Model':<25} {'Acc':<6} {'Prec':<6} {'Rec':<6} {'F1':<6} {'FNR':<6} {'Safety'}")
    print("-"*80)
    
    for scenario in scenarios:
        name = scenario['name'].split(':')[0]
        safety = '✅ GOOD' if scenario['fnr'] < 0.05 else '❌ BAD'
        print(f"{name:<25} {scenario['accuracy']:<6.3f} {scenario['precision']:<6.3f} "
              f"{scenario['recall']:<6.3f} {scenario['f1']:<6.3f} "
              f"{scenario['fnr']:<6.3f} {safety}")
    
    print("\n" + "="*80)
    print("\n🎯 Key Insights for AV Perception:")
    print("\n1. High ACCURACY ≠ Safe for AVs!")
    print("   • Model B has 95% accuracy but 10% FNR (misses 1 in 10 pedestrians!)")
    print("\n2. RECALL is critical for safety:")
    print("   • False negatives = missed pedestrians = potential fatalities")
    print("   • Model C: 98% recall, only 2% FNR ✅")
    print("\n3. Trade-off: Recall vs Precision")
    print("   • High recall → More false alarms (uncomfortable braking)")
    print("   • But this is MUCH better than missing pedestrians!")
    print("\n4. Target metrics for pedestrian detection:")
    print("   • Recall: >98% (FNR <2%)")
    print("   • Precision: >90% (acceptable false alarm rate)")
    print("   • F1-score: >0.93")
    print("\n💡 For safety: Optimize for RECALL first, then improve precision")

safety_metric_comparison()

## 6. Complete Validation Plan Template

Let's create a comprehensive validation plan for an AV perception system.

In [ ]:
def create_validation_plan():
    """
    Create a comprehensive validation plan template.
    """
    
    plan = {
        'System': 'AV Perception System (Pedestrian Detection)',
        'ODD': 'Urban environment, <50 km/h, daylight and dusk, dry and light rain',
        
        'Phases': [
            {
                'phase': '1. SIL (Simulation)',
                'scenarios': 1000000,
                'duration': '3 months',
                'objectives': [
                    'Cover full parameter space',
                    'Test all weather/lighting combinations',
                    'Verify functional requirements',
                    'Measure baseline performance'
                ],
                'acceptance': [
                    'Recall >98% across all scenarios',
                    'Precision >85%',
                    'ECE <0.05',
                    'Coverage >95% of parameter space'
                ]
            },
            {
                'phase': '2. HIL (Hardware-in-Loop)',
                'scenarios': 10000,
                'duration': '2 months',
                'objectives': [
                    'Validate on real hardware',
                    'Test real-time performance',
                    'Focus on critical scenarios',
                    'Verify sensor fusion'
                ],
                'acceptance': [
                    'Recall >97% on critical scenarios',
                    'Latency <100ms (95th percentile)',
                    'No regression from SIL',
                    'Hardware meets specifications'
                ]
            },
            {
                'phase': '3. VIL (Proving Ground)',
                'scenarios': 1000,
                'duration': '2 months',
                'objectives': [
                    'Test complete system integration',
                    'Validate dangerous scenarios safely',
                    'Test edge cases',
                    'Verify fallback behaviors'
                ],
                'acceptance': [
                    'Zero missed detections in critical scenarios',
                    'Successful emergency braking tests',
                    'Correct uncertainty estimates',
                    'All edge cases handled'
                ]
            },
            {
                'phase': '4. FOT (Field Tests)',
                'scenarios': '100 critical + 10,000 miles general',
                'duration': '6 months',
                'objectives': [
                    'Validate in real-world conditions',
                    'Discover unknown scenarios',
                    'Build statistical evidence',
                    'Final confidence before deployment'
                ],
                'acceptance': [
                    'Zero safety-critical failures',
                    'Interventions <1 per 1000 miles',
                    'All unknowns properly handled',
                    'Regulatory approval achieved'
                ]
            }
        ],
        
        'Metrics': {
            'Primary': [
                'Recall (>98%)',
                'False Negative Rate (<2%)',
                'Detection range (>30m urban)'
            ],
            'Secondary': [
                'Precision (>90%)',
                'F1-score (>0.93)',
                'Latency (<100ms, p95)'
            ],
            'Safety': [
                'Calibration ECE (<0.05)',
                'OOD detection rate (>95%)',
                'Worst-case performance'
            ]
        },
        
        'Risk_Mitigation': [
            'Diverse test scenarios including edge cases',
            'Progressive validation (SIL→HIL→VIL→FOT)',
            'Uncertainty monitoring in production',
            'Continuous data collection and retraining',
            'Redundant perception (multiple sensors)',
            'Human-in-loop for unknown scenarios'
        ]
    }
    
    # Print plan
    print("\n" + "="*80)
    print(f"VALIDATION PLAN: {plan['System']}")
    print("="*80)
    print(f"\nOperational Design Domain (ODD):")
    print(f"  {plan['ODD']}")
    
    print("\n" + "-"*80)
    print("VALIDATION PHASES")
    print("-"*80)
    
    for phase_data in plan['Phases']:
        print(f"\n{phase_data['phase']}")
        print(f"  Scenarios: {phase_data['scenarios']:,}" if isinstance(phase_data['scenarios'], int) 
              else f"  Scenarios: {phase_data['scenarios']}")
        print(f"  Duration: {phase_data['duration']}")
        print(f"\n  Objectives:")
        for obj in phase_data['objectives']:
            print(f"    • {obj}")
        print(f"\n  Acceptance Criteria:")
        for criterion in phase_data['acceptance']:
            print(f"    ✓ {criterion}")
    
    print("\n" + "-"*80)
    print("VALIDATION METRICS")
    print("-"*80)
    
    for category, metrics in plan['Metrics'].items():
        print(f"\n{category} Metrics:")
        for metric in metrics:
            print(f"  • {metric}")
    
    print("\n" + "-"*80)
    print("RISK MITIGATION STRATEGIES")
    print("-"*80)
    for strategy in plan['Risk_Mitigation']:
        print(f"  ✓ {strategy}")
    
    print("\n" + "="*80)
    print("\nTotal timeline: ~13 months")
    print("Total cost estimate: $2-5M (depending on simulator, proving ground, fleet)")
    print("\nThis plan provides ISO 26262 and ISO 21448 (SOTIF) compliant evidence.")
    print("="*80)
    
    return plan

validation_plan = create_validation_plan()

## Summary and Key Takeaways

### What We Learned

1. **Validation Challenges:**
   - Cannot test all possible scenarios
   - Pure statistical proof requires millions of miles
   - Need multi-faceted approach

2. **Scenario-Based Testing:**
   - Pegasus 6-layer model: concrete → functional → logical → ODD
   - Systematically cover parameter space
   - Focus on critical/corner cases

3. **Simulation + Real-World:**
   - SIL: Millions of scenarios, full coverage
   - HIL: Thousands of scenarios, real hardware
   - VIL: Hundreds of scenarios, complete system
   - FOT: Targeted scenarios + general driving

4. **Statistical Evidence:**
   - Need ~275M miles to prove better than humans (impractical!)
   - Use surrogate metrics and simulation
   - Continuous monitoring in operation

5. **Safety Metrics:**
   - Prioritize RECALL over precision
   - False negatives are safety-critical
   - Measure worst-case performance
   - Validate uncertainty estimates

### Best Practices

1. **Progressive validation:** Simulation → Proving ground → Public roads
2. **Diverse scenarios:** Cover full ODD systematically
3. **Safety-first metrics:** Optimize for recall, not just accuracy
4. **Uncertainty monitoring:** Track model confidence in production
5. **Continuous learning:** Collect edge cases, retrain, re-validate

---

## Congratulations!

You've completed **Session 4: Uncertainty Estimation and Validation**!

You now understand:
- ✅ Uncertainty types and quantification methods
- ✅ MC Dropout and Deep Ensembles
- ✅ Model calibration techniques
- ✅ Comprehensive validation strategies

**This completes the AV Perception Safety Workshop!**

### Next Steps

- Complete Exercise 7: Uncertainty Quantification
- Complete Exercise 8: Design a Validation Strategy
- Apply these techniques to your own AV projects
- Stay updated: This field is rapidly evolving!